In [28]:
'''Mask, No mask dataset

https://drive.google.com/drive/folders/1XDte2DL2Mf_hw4NsmGst7QtYoU7sMBVG
'''

'Mask, No mask dataset\n\nhttps://drive.google.com/drive/folders/1XDte2DL2Mf_hw4NsmGst7QtYoU7sMBVG\n'

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
dataset= r'C:\Users\jainrohi\Desktop\imageDL\face detector\dataset-20200917T100622Z-001\dataset'

In [3]:
imagePaths=list(paths.list_images(dataset))

In [4]:
imagePaths[:5],imagePaths[-5:]

(['C:\\Users\\jainrohi\\Desktop\\imageDL\\face detector\\dataset-20200917T100622Z-001\\dataset\\without_mask\\0.jpg',
  'C:\\Users\\jainrohi\\Desktop\\imageDL\\face detector\\dataset-20200917T100622Z-001\\dataset\\without_mask\\0_0_aidai_0014.jpg',
  'C:\\Users\\jainrohi\\Desktop\\imageDL\\face detector\\dataset-20200917T100622Z-001\\dataset\\without_mask\\0_0_aidai_0029.jpg',
  'C:\\Users\\jainrohi\\Desktop\\imageDL\\face detector\\dataset-20200917T100622Z-001\\dataset\\without_mask\\0_0_aidai_0043.jpg',
  'C:\\Users\\jainrohi\\Desktop\\imageDL\\face detector\\dataset-20200917T100622Z-001\\dataset\\without_mask\\0_0_aidai_0074.jpg'],
 ['C:\\Users\\jainrohi\\Desktop\\imageDL\\face detector\\dataset-20200917T100622Z-001\\dataset\\with_mask\\with_mask655.jpeg',
  'C:\\Users\\jainrohi\\Desktop\\imageDL\\face detector\\dataset-20200917T100622Z-001\\dataset\\with_mask\\with_mask656.jpeg',
  'C:\\Users\\jainrohi\\Desktop\\imageDL\\face detector\\dataset-20200917T100622Z-001\\dataset\\with_ma

In [5]:
from tqdm import tqdm

In [6]:
data=[]
labels=[]

for i in tqdm(imagePaths):
    label=i.split(os.path.sep)[-2]
    labels.append(label)
    image=load_img(i,target_size=(224,224))
    image=img_to_array(image)
    image=preprocess_input(image)
    data.append(image)

 83%|███████████████████████████████████████████████████████████████▋             | 3173/3833 [00:07<00:01, 384.18it/s]C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████| 3833/3833 [00:38<00:00, 98.97it/s]


In [11]:
data[0], labels[:5], labels[-5:]

(array([[[-0.94509804, -0.9764706 , -0.85882354],
         [-0.94509804, -0.9764706 , -0.85882354],
         [-0.94509804, -0.9764706 , -0.85882354],
         ...,
         [-0.8509804 , -0.90588236, -0.78039217],
         [-0.8509804 , -0.90588236, -0.78039217],
         [-0.84313726, -0.8980392 , -0.77254903]],
 
        [[-0.9137255 , -0.94509804, -0.827451  ],
         [-0.9137255 , -0.94509804, -0.827451  ],
         [-0.9137255 , -0.94509804, -0.827451  ],
         ...,
         [-0.7647059 , -0.79607844, -0.6784314 ],
         [-0.7647059 , -0.79607844, -0.6784314 ],
         [-0.75686276, -0.7882353 , -0.67058825]],
 
        [[-0.92156863, -0.9607843 , -0.8039216 ],
         [-0.92156863, -0.9607843 , -0.8039216 ],
         [-0.92156863, -0.9607843 , -0.8039216 ],
         ...,
         [-0.88235295, -0.8980392 , -0.7882353 ],
         [-0.8666667 , -0.88235295, -0.77254903],
         [-0.85882354, -0.8745098 , -0.7647059 ]],
 
        ...,
 
        [[ 0.3803922 , -0.9607843 

In [12]:
data = np.array(data,dtype='float32')
labels = np.array(labels)

In [13]:
data.shape

(3833, 224, 224, 3)

In [14]:
lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

In [17]:
labels[0]

array([0., 1.], dtype=float32)

In [18]:
train_X,test_X,train_Y,test_Y=train_test_split(data,labels,test_size=0.20,stratify=labels,random_state=10)
train_X.shape, train_Y.shape, test_X.shape, test_Y.shape

((3066, 224, 224, 3), (3066, 2), (767, 224, 224, 3), (767, 2))

In [19]:
aug=ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,vertical_flip=True,fill_mode='nearest')

In [20]:
baseModel=MobileNetV2(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))

9412608/9406464 [==============================] - 6s 1us/step


In [21]:
baseModel.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [22]:
headModel=baseModel.output
print(headModel)
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name='Flatten')(headModel)
headModel=Dense(128,activation='relu')(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation='softmax')(headModel)

model = Model(inputs=baseModel.input,outputs=headModel)

Tensor("out_relu/Relu6:0", shape=(None, 7, 7, 1280), dtype=float32)


In [23]:
for layer in baseModel.layers:
    layer.trainable=False

In [25]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [26]:
learning_rate=0.001
Epochs=20
BS=12

opt=Adam(lr=learning_rate,decay=learning_rate/Epochs)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

H=model.fit(
    aug.flow(train_X,train_Y,batch_size=BS),
    steps_per_epoch=len(train_X)//BS,
    validation_data=(test_X,test_Y),
    validation_steps=len(test_X)//BS,
    epochs=Epochs
)


model.save(r'C:\Users\jainrohi\Desktop\imageDL\face detector\mobile-net-v2-my-trained-model.model')

Epoch 1/20
255/255 [==============================] - 271s 1s/step - loss: 0.1670 - accuracy: 0.9342 - val_loss: 0.0634 - val_accuracy: 0.9765
Epoch 2/20
255/255 [==============================] - 218s 855ms/step - loss: 0.1008 - accuracy: 0.9610
Epoch 3/20
255/255 [==============================] - 224s 877ms/step - loss: 0.0944 - accuracy: 0.9663
Epoch 4/20
255/255 [==============================] - 219s 859ms/step - loss: 0.0678 - accuracy: 0.9761
Epoch 5/20
255/255 [==============================] - 213s 836ms/step - loss: 0.0704 - accuracy: 0.9751
Epoch 6/20
255/255 [==============================] - 222s 870ms/step - loss: 0.0618 - accuracy: 0.9790
Epoch 7/20
255/255 [==============================] - 217s 850ms/step - loss: 0.0587 - accuracy: 0.9800
Epoch 8/20
255/255 [==============================] - 201s 789ms/step - loss: 0.0520 - accuracy: 0.9813
Epoch 9/20
255/255 [==============================] - 217s 852ms/step - loss: 0.0487 - accuracy: 0.9826
Epoch 10/20
255/255 [====

In [27]:
predict=model.predict(test_X,batch_size=BS)
predict=np.argmax(predict,axis=1)
print(classification_report(test_Y.argmax(axis=1),predict,target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       383
without_mask       0.99      0.99      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767

